# Generate Data for Chartbook

Brian Dew

@bd_econ

In [1]:
import sys
import json
sys.path.append('../src')

import requests

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

In [2]:
#nipa_series_codes(retrieve_table('T11000'))

### Labor Share

In [2]:
s = ['A261RC', 'A4002C']

df = nipa_df(retrieve_table('T11000')['Data'], s).sort_index()
df['Share'] = (df['A4002C'] / df['A261RC']) * 100
data = df.loc['1989':, 'Share'].dropna()
data.to_csv(data_dir / 'laborshare.csv', 
            index_label='date', 
            float_format='%g')

node = end_node(data, 'blue!60!cyan')
write_txt(text_dir / 'laborshare_node.txt', node)

In [3]:
s = series_info(data)
ltdate = dtxt(s['date_latest'])['qtr2']
one_yr = val_inc_pp(s['val_latest'] - s['val_year_ago'])
ltmin = s['val_latest'] - s['val_min']
ltmax = s['val_max'] - s['val_latest']
text = (f'As of {ltdate}, labor receives '+
        f'{s["val_latest"]:.1f} percent of gross domestic income, '+
        f'and the labor share {one_yr} over the past year. '+
        f'The labor share is {ltmin:.1f} percentage '+
        f'points above its 30-year low of {s["val_min"]:.1f} percent '+
        f'in {s["date_min_ft"]}, but {ltmax:.1f} percentage points '+
        f'below the 30-year high of {s["val_max"]:.1f} percent in '+
        f'{s["date_max_ft"]}. ')

write_txt(text_dir / 'laborshare.txt', text)
text

'As of the fourth quarter of 2019, labor receives 53.5 percent of gross domestic income, and the labor share increased by a total of 0.4 percentage points over the past year. The labor share is 1.9 percentage points above its 30-year low of 51.6 percent in 2014 Q3, but 3.9 percentage points below the 30-year high of 57.4 percent in 1992 Q3. '

### GDP growth rate

In [4]:
s = ['A191RL']

df = nipa_df(retrieve_table('T10502')['Data'], s).sort_index()

df.loc['1989':].to_csv(data_dir / 'gdp.csv', index_label='date')

date = dtxt(df.index[-1])['qtr1']

txt = f'{date}: {df["A191RL"].iloc[-1]}\%'

write_txt(data_dir / 'gdp.txt', txt)

### Private fixed investment

In [5]:
# Private fixed investment contribution to growth
s = ['A008RY', 'A011RY', 'A014RY']

df = nipa_df(retrieve_table('T10502')['Data'], s)

df.loc['1989':].to_csv(data_dir / 'inv.csv', index_label='date')

ldate = df.index[-1]
datetext = dtxt(ldate)["qtr1"]

tot_contr = cont_subt(df.loc[ldate, ['A008RY', 'A011RY']].sum())

bus_contr = cont_subt(df.loc[ldate, 'A008RY'], 'end')

res_contr = cont_subt(df.loc[ldate, 'A011RY'], 'end')

inv_contr = cont_subt(df.loc[ldate, 'A014RY'], 'end')

text = (f'During the quarter, private fixed investment {tot_contr} real GDP growth. '+
        f'Non-residential fixed investment {bus_contr}, while '+
        f'residential fixed investment {res_contr}. The change in private '+
        f'inventories {inv_contr}.')

write_txt(text_dir / 'inv_text2.txt', text)

In [6]:
# In levels and share of GDP
s = ['A191RC', 'A007RC', 'A008RC', 'A011RC']

df = nipa_df(retrieve_table('T10105')['Data'], s)

ldate = df.index[-1]

inv_tot = df.loc[ldate, 'A007RC'] / 1_000_000

gdp_tot = df.loc[ldate, 'A191RC'] / 1_000_000

res_tot = df.loc[ldate, 'A011RC'] / 1_000

bus_tot = df.loc[ldate, 'A008RC'] / 1_000_000

inv_sh = (inv_tot / gdp_tot) * 100

res_sh = (res_tot / 1_000 / gdp_tot) * 100

bus_sh = (bus_tot / gdp_tot) * 100

text = (f'In {dtxt(ldate)["qtr2"]}, private fixed investment, '+
        f'which does not include inventory investment, totals \${inv_tot:.1f} trillion, '+
        f'equivalent to {inv_sh:.1f} percent of GDP. Non-residential (business) fixed '+
        f'investment totals \${bus_tot:.1f} trillion, or {bus_sh:.1f} percent of GDP, '+
        f'while residential fixed investment totals \${res_tot:.1f} billion ({res_sh:.1f} '+
        'percent of GDP).')

write_txt(text_dir / 'inv_text.txt', text)

### Imports and Exports

In [7]:
# Import and Export share of GDP
s = ['B020RC', 'B021RC', 'B648RC', 'LA000006']
s2 = ['A191RC']
df = nipa_df(retrieve_table('T40205')['Data'], s)
df['A191RC'] = nipa_df(retrieve_table('T10105')['Data'], s2)
df['EX'] = df['B020RC'] - df['LA000006']
df['IM'] = df['B021RC'] - df['B648RC']
data = df.div(df['A191RC'], axis=0) * 100
data.loc['1989':].to_csv(data_dir / 'eximgdp.csv', index_label='date')

date = f'{qtrs[data.index[-1].quarter]} quarter of {data.index[-1].year}'
valex = data['EX'].iloc[-1]
valim = data['IM'].iloc[-1]

text = (f'Nonpetroleum goods and services imports (see {{\color{{green!60!teal!80!black}}'+
        f'\\textbf{{---}}}}) were equivalent to {valim:.1f} percent of GDP in the {date}, '+
        f'while exports of nonpetroleum goods and services (see {{\\color{{blue!90!cyan}}'+
        f'\\textbf{{---}}}}) were equivalent to {valex:.1f} percent of GDP.')

write_txt(text_dir / 'exim.txt', text)

text

'Nonpetroleum goods and services imports (see {\\color{green!60!teal!80!black}\\textbf{---}}) were equivalent to 13.2 percent of GDP in the fourth quarter of 2019, while exports of nonpetroleum goods and services (see {\\color{blue!90!cyan}\\textbf{---}}) were equivalent to 10.6 percent of GDP.'

### Goods Import Penetration

In [17]:
s = ['A353RC']

G = nipa_df(retrieve_table('T10205')['Data'], s).sort_index()

s = ['A253RC', 'A255RC', 'B647RC', 'LA000004', 'A650RC', 'B651RC', 'A652RC', 'A653RC', 'B648RC']

MX = nipa_df(retrieve_table('T40205')['Data'], s).sort_index()

D = G['A353RC'] - MX['A253RC'] + MX['A255RC']
result = (MX['A255RC'] / D)

import_categories = ['B647RC', 'LA000004', 'A650RC', 'B651RC', 
                     'A652RC', 'A653RC', 'B648RC']
Msh = MX[import_categories].div(MX['A255RC'], axis=0)

Msh['Consumer'] = Msh['B647RC'] + Msh['A652RC'] + Msh['B651RC']
Msh['Capital'] = Msh['LA000004'] - Msh['B648RC'] + Msh['A650RC'] + Msh['A653RC']

final = Msh[['Consumer', 'Capital', 'B648RC']].multiply(result, axis=0) * 100

final.loc['1989':].to_csv(data_dir / 'goodsimpsh.csv', index_label='date')

ltdate = final.index[-1]
datetext = dtxt(ltdate)['qtr1']
cons = final['Consumer'].iloc[-1]
capi = final['Capital'].iloc[-1]
oil = final['B648RC'].iloc[-1]

text = (f'As of {datetext}, imports of consumer goods excluding petroleum '+
        f'and petroleum products are equivalent to {cons:.1f} percent of domestic '+
        'consumption of goods (see\cbox{cyan!40!white}). Petroleum-related imports '+
        f'claim {oil:.1f} percent (see\cbox{{purple}}) and imports of all other goods, '+
        f'primarily capital goods, industrial supplies, and materials are equivalent to '+
        f'{capi:.1f} percent (see\cbox{{blue!50!cyan}}).')

write_txt(text_dir / 'goodsimpsh1.txt', text)

ch11 = (final.loc['2011-01-01'] - final.iloc[0])

chlt = (final.iloc[-1] - final.loc['2011-01-01'])

chtxt = {}
for i, v in chlt.items():
    if v >= 0.1:
        chtxt[i] = f'increased by the equivalent of {v:.1f} percent'
    elif v <= -0.1:
        chtxt[i] = f'decreased by the equivalent of {abs(v):.1f} percent'
    else:
        chtxt[i] = 'was virtually unchanged'
        
text = ('From 1989 to 2011, imports of consumer goods excluding petroleum increased '+
        f'by the equivalent of {ch11["Consumer"]:.1f} percent of domestic '+
         'consumption of goods; petroleum and products imports '+
        f'increased by the equilavent of {ch11["B648RC"]:.1f} percent; '+
        'and all other goods increased by the equivalent '+
        f'of {ch11["Capital"]:.1f} percent. Since 2011, imports of '+
        f'consumer goods {chtxt["Consumer"]} of domestic goods demand; '+
        f'imports of petroleum and products {chtxt["B648RC"]}; and other '+
        f'imports {chtxt["Capital"]}.')

write_txt(text_dir / 'goodsimpsh.txt', text)

### ITA - Financial Account Balance

In [10]:
def bea_api_ita(ind_list, bea_key):
    ''' Return tables in table list for years in range'''
    import requests
    from datetime import datetime

    years = ','.join(map(str, range(1988, 2020)))

    api_results = []

    for ind in ind_list:
        url = f'https://www.bea.gov/api/data/?&UserID={bea_key}'\
              f'&method=GetData&datasetname=ITA&Indicator={ind}'\
              f'&Frequency=QSA&Year={years}&ResultFormat=json'

        r = requests.get(url)

        api_results.append((ind, r.text))

    return api_results

In [11]:
ind_list = ['FinAssetsExclFinDeriv', 'FinLiabsExclFinDeriv', 'FinDeriv', 'StatDisc']

api_results = bea_api_ita(ind_list, bea_key)

gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']

results = pd.DataFrame({name: {i['TimePeriod']: i['DataValue'] 
                               for i in json.loads(series)['BEAAPI']['Results']['Data']} 
                        for name, series in api_results})
results.index = pd.to_datetime(results.index)
results = results.replace(r'^\s*$', np.nan, regex=True).astype('float').rolling(4).sum()
results['FAB'] = results['FinLiabsExclFinDeriv'] - results['FinAssetsExclFinDeriv']
results['TOT'] = results[['FAB', 'StatDisc', 'FinDeriv']].sum(axis=1)
final = (results.divide(gdp, axis=0).dropna(how='all') * 100).fillna(0)
final.loc['1989':, ['FAB', 'StatDisc', 'FinDeriv', 'TOT']].to_csv(data_dir / 'fab.csv', index_label='date')

s = final.iloc[-1]
liab = s.FinLiabsExclFinDeriv
assets = s.FinAssetsExclFinDeriv

ldate = dtxt(final.index[-1])['qtr1']

text = (f'Over the year ending {ldate}, net domestic acquisitions of foreign assets were '+
        f'equivalent to {assets:.1f} percent of GDP, while net domestic incurrence of foreign '+
        f'liabilities total {liab:.1f} percent of GDP. Domestic net borrowing totals '+
        f'{s.TOT:.1f} percent of GDP.')

write_txt(text_dir / 'fab.txt', text)

text

'Over the year ending 2019 Q4, net domestic acquisitions of foreign assets were equivalent to 2.0 percent of GDP, while net domestic incurrence of foreign liabilities total 3.6 percent of GDP. Domestic net borrowing totals 1.9 percent of GDP.'

### GDP Composition

In [12]:
s = ['DPCERY', 'A006RY', 'A822RY', 'A019RY']

(nipa_df(retrieve_table('T10502')['Data'], s).loc['1989':]
 .to_csv(data_dir / 'comp.csv', index_label='date'))

### GDP per capita

In [13]:
s = ['A939RC', 'A939RX']

df = nipa_df(retrieve_table('T70100')['Data'], s)
df['value'] = (df['A939RX'] / df['A939RX'].iloc[-1])  * df['A939RC'].iloc[-1]
df[['value']].loc['1989':].to_csv(data_dir / 'gdppc.csv', index_label='date')

### Domestic Investment

In [14]:
s = ['A261RX', 'W256RX']
rgdi = nipa_df(retrieve_table('T11706')['Data'], s).dropna()

s = ['A261RC', 'A4002C', 'W056RC', 'A107RC', 'W271RC', 'A262RC']
df = nipa_df(retrieve_table('T11000')['Data'], s).dropna()

# Calculate indirect taxes net of transfers
df['indirect'] = df['W056RC'] - df['A107RC']
df = df.drop(['A107RC', 'W056RC'], axis=1)

# Calculate GDI deflator from real GDI series
deflator = rgdi['A261RX'] / df['A261RC']
deflator = deflator / deflator.iloc[-1]
df = df.multiply(deflator, axis=0)

# Calculate contributions to growth
dft = df.diff()
dft = dft.div(dft['A261RC'], axis=0)
contr = dft.multiply((((df['A261RC'].pct_change() + 1) ** 4) - 1) * 100, axis=0)
contr.loc['1989':].to_csv(data_dir / 'gdi.csv', index_label='date')

### Consumer Spending

In [15]:
s = ['DGDSRY', 'DHCERY', 'DHUTRY', 'A011RY']

(nipa_df(retrieve_table('T10502')['Data'], s)
 .assign(OTHSER = lambda x: x['DHCERY'] - x['DHUTRY'],
         HOUSING = lambda x: x['A011RY'] + x['DHUTRY'])
 .drop('DHCERY', axis=1).loc['1989':]
 .to_csv(data_dir / 'pce.csv', index_label='date', float_format='%g'))

### Trade Contribution to GDP

In [16]:
s = ['A019RY', 'A253RY', 'A646RY', 'A255RY', 'A656RY']
df = nipa_df(retrieve_table('T10502')['Data'], s)
df.loc['1989':].to_csv(data_dir / 'nx.csv', index_label='date')

In [17]:
sl = [('A253RY', 'main'), ('A646RY', 'end'), 
      ('A255RY', 'main'), ('A656RY', 'end')]

d = {}

for s, style in sl:
    # Latest total value
    value = df[s].iloc[-1]
    d[s] = cont_subt(value, style=style)

# Text for household expenditure section
q = {1: 'first', 2: 'second', 3: 'third', 4: 'fourth'}
    
ldate = dtxt(df.index[-1])['qtr2']


text = (f"Goods exports {d['A253RY']} GDP growth in {ldate} while "+
        f"services exports {d['A646RY']}. Good imports {d['A255RY']} "+
        f"GDP growth and services imports {d['A656RY']}.")

write_txt(text_dir / 'trade.txt', text)

### Business investment

In [18]:
s = ['Y001RY', 'A009RY', 'Y033RY']

df = nipa_df(retrieve_table('T10502')['Data'], s)
df.loc['1989':].to_csv(data_dir / 'businv.csv', index_label='date')

### Durable goods new orders

In [19]:
# New orders for capital goods excluding defense or aircraft
key = census_key
base = 'https://api.census.gov/data/timeseries/eits/advm3'
param = 'cell_value,time_slot_id'
t = '&time=from+1992'
cat = '&category_code=NXA'
dtc = '&data_type_code=NO'
oth = '&for=us&seasonally_adj=yes'

url = f'{base}?get={param}&key={key}{cat}{dtc}{t}{oth}'

r = requests.get(url).json()

df = pd.DataFrame({'date': [pd.to_datetime(i[4]) for i in r[1:]], 
                   'value': [float(i[0]) for i in r[1:]]}).sort_values('date')

df = df.set_index('date')

gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])

result = ((df['value'].resample('QS').sum() * 4  / gdp['A191RC']).dropna() * 100).iloc[1:]
(result.rename('value').loc['1989':].to_csv(data_dir / 'dgno.csv', index_label='date', header=True))

node = end_node(result, 'purple!50!violet')
write_txt(text_dir / 'dgno_node.txt', node)


ldate = dtxt(df.index[-1])['mon1']
date_latest = dtxt(df.index[-1])['datetime']
month_short = df.index[-1].strftime('%b')

In [21]:
one_yr_ch = (df.pct_change(12) * 100).dropna()['value'].iloc[-1]

if one_yr_ch >= 0.1:
    val = f'increased by {one_yr_ch:.1f} percent'
elif one_yr_ch <= -0.1:
    val = f'decreased by {abs(one_yr_ch):.1f} percent'
else:
    val = 'were virtually unchanged'

text = ('New orders for manufactured core capital goods excluding aircraft '+
        f'totalled \${df.iloc[-1][0] / 1000:,.0f} billion in {ldate}, '+
        f'equivalent to {result.iloc[-1]:.1f} percent of GDP '+
        '(see {\color{purple!50!violet}\\textbf{---}}). New orders '+
        f'{val} over the past year.')

write_txt(text_dir / 'dgno.txt', text)

text

'New orders for manufactured core capital goods excluding aircraft totalled \\$69 billion in February 2020, equivalent to 3.8 percent of GDP (see {\\color{purple!50!violet}\\textbf{---}}). New orders decreased by 0.6 percent over the past year.'

In [22]:
text = ('xtick={{1992-01-01}, {1995-01-01}, {2000-01-01}, {2005-01-01}, '+
f'{{2010-01-01}}, {{2015-01-01}}, {{{date_latest}}}}}, '+
f'xticklabels={{`92, `95, `00, `05, `10, `15, {month_short}}}, ')

text_full = ('\\begin{tikzpicture}'+
'\\begin{axis}[\\bbar{y}{0}, \dateaxisticks ytick={4, 6, 8}, width=6.5cm, height=5.4cm,'+
'ymin=2.9, clip=false,'+
text + 
'minor xtick={}]'+
'\\rebars'+
'\\thickline{purple!50!violet}{date}{value}{data/dgno.csv}'+
'\input{text/dgno_node.txt}'+
'\end{axis}'+
'\end{tikzpicture}')

write_txt(text_dir / 'dgno.tex', text_full)

### Retail sales

In [23]:
# New orders for capital goods excluding defense or aircraft
key = census_key
base = 'https://api.census.gov/data/timeseries/eits/marts/'
param = 'cell_value,time_slot_id,category_code'
t = '&time=from+1992'
dtc = '&data_type_code=SM'
oth = '&for=us&seasonally_adj=yes'

url = f'{base}?get={param}&key={key}{dtc}{t}{oth}'

r = requests.get(url).json()

In [24]:
df = pd.DataFrame()
for series in ['44000', '44X72', '44W72', '454']:
    df[series] = pd.Series(
        {pd.to_datetime(i[4]): 
         float(i[0]) for i in r[1:] if i[2] == series}
    ).sort_index()
    
data = (df.pct_change(12) * 100).dropna()
data['NS_3M'] = data['454'].rolling(3).mean()

s = ['A191RC']

gdp = nipa_df(retrieve_table('T10105')['Data'], s)

data.to_csv(data_dir / 'marts.csv', index_label='date', float_format='%g')

node = end_node(data['NS_3M'], 'blue!70!black')
write_txt(text_dir / 'rs_ns3m_node.txt', node)

node = end_node(data['44X72'], 'green!90!blue')
write_txt(text_dir / 'rs_44x72_node.txt', node)

totval = df['44X72'].iloc[-1]
shgdp = (totval * 12 / gdp.iloc[-1] * 100)[-1]
totgr = data['44X72'].iloc[-1]
datelt = df.index[-1].strftime('%B %Y')
grtxt = inc_dec_percent(totgr)
    
totval2 = df['454'].iloc[-1]
shgdp2 = (totval2 * 12 / gdp.iloc[-1] * 100)[-1]
totgr2 = data['454'].iloc[-1]
grtxt2 = inc_dec_percent(totgr2)
    
text = ('According to the \href{https://www.census.gov/retail/index.html}{Census Bureau}, '+
        'retail and food service '+
        f'sales totalled \${totval/1000:,.1f} billion in {datelt}, equivalent '+
        f'to roughly {shgdp:.1f} percent of GDP on an annualized basis. '+
        'Over the past year, retail and '+
        f'food service sales {grtxt}, without adjusting for prices '+
        '(see {\color{green!90!blue}\\textbf{---}}). Nonstore '+
        f'sales, which include online retailers, have {grtxt2} over '+
        'the same period (see {\color{blue!70!black}\\textbf{---}}), and '
        f'total \${totval2/1000:,.1f} billion, or roughly {shgdp2:.1f} percent of GDP. ')

write_txt(text_dir / 'marts.txt', text)

text

'According to the \\href{https://www.census.gov/retail/index.html}{Census Bureau}, retail and food service sales totalled \\$528.1 billion in February 2020, equivalent to roughly 29.2 percent of GDP on an annualized basis. Over the past year, retail and food service sales increased by 4.3 percent, without adjusting for prices (see {\\color{green!90!blue}\\textbf{---}}). Nonstore sales, which include online retailers, have increased by 7.5 percent over the same period (see {\\color{blue!70!black}\\textbf{---}}), and total \\$66.6 billion, or roughly 3.7 percent of GDP. '

### Retail Sales by Type

In [25]:
key = census_key
base = 'https://api.census.gov/data/timeseries/eits/marts/'
param = 'cell_value,time_slot_id,category_code'
t = '&time=from+1992'
dtc = '&data_type_code=SM'
oth = '&for=us&seasonally_adj=no'

url = f'{base}?get={param}&key={key}{dtc}{t}{oth}'

r = requests.get(url).json()

In [26]:
series_dict = {'441': 'Motor Vehicles \& Parts', '442': 'Furniture \& Home Furnishings', 
               '443': 'Electronics \& Appliance', '444': 'Building \& Garden Equipment', 
               '445': 'Food \& Beverage Stores', '446': 'Health \& Personal Care', 
               '447': 'Gasoline Stations', '448': 'Clothing and Accessories', 
               '451': 'Sports/Hobby/Music/Books', '452': 'General Merchandise', 
               '454': 'Nonstore', '722': 'Food Service \& Drinking Places'}

df = pd.DataFrame()
for series in series_dict.keys():
    df[series] = pd.Series(
        {pd.to_datetime(i[4]): 
         float(i[0]) for i in r[1:] if i[2] == series}
    ).sort_index()
    
dpi = nipa_df(retrieve_table('T20100')['Data'], ['A067RC'])['A067RC']
data = (df.resample('QS').sum().rolling(4).sum()
          .dropna().divide(dpi, axis=0).dropna() * 100)
results = (pd.concat([data.loc['2015-01-01'], data.loc['2019-10-01']], axis=1)
             .sort_values('2019-10-01', ascending=False))
results.index = results.index.map(series_dict)
results.round(2).to_csv(data_dir / 'rs_comp.csv', index_label='name')

### Government spending and investment

In [18]:
n = {'A822RY': 'Total',
     'A823RY': '\hspace{1mm}Federal total',
     'A824RY': '\hspace{1mm}\cbox{blue!60!black}National defense',
     'A997RY': '\hspace{7mm}Consumption expenditures',
     'A788RY': '\hspace{7mm}Gross investment',
     'A825RY': '\hspace{1mm}\cbox{green!85!black}Nondefense',
     'A542RY': '\hspace{7mm}Consumption expenditures',
     'A798RY': '\hspace{7mm}Gross investment',
     'A829RY': '\hspace{-2mm}\cbox{purple!70!magenta}State \& local',
     'A991RY': '\hspace{5mm}Consumption expenditures',
     'A799RY': '\hspace{5mm}Gross investment'}

s = n.keys()

df = nipa_df(retrieve_table('T10502')['Data'], s)
df.loc['1989':].to_csv(data_dir / 'gov.csv', index_label='date')

d = series_info(df['A822RY'])

if d['val_latest'] > 0:
    text1 = f'contributed {d["val_latest"]} percentage points to' 
elif d['val_latest'] < 0:
    text1 = f'subtracted {abs(d["val_latest"])} percentage points to'
else:
    text1 = 'did not contribute to'
    
d2 = {}
for i in ['A824RY', 'A825RY', 'A829RY']:
    if df[i].iloc[-1] > 0:
        ctxt = f'contributed {df[i].iloc[-1]:.2f} percentage points'
    elif df[i].iloc[-1] < 0:
        ctxt = f'subtracted {abs(df[i].iloc[-1]):.2f} percentage points'
    else:
        ctxt = 'did not contribute '
    d2[i] = ctxt
    
text = 'Government consumption expeditures and gross investment, which provide services and infrastructure, '

gov = (f'{text}{text1} real GDP growth in {d["date_latest_ft"]}, compared to an '+
       f'average contribution of {d["one_year_mean"]:.2f} percentage points over '+
       f'the past year and an average of {d["mean"]:.2f} percentage points since 1989. '+
       f'In {d["date_latest_ft"]}, federal defense (see\cbox{{blue!60!black}}) {d2["A824RY"]}, '+
       f'federal nondefense (see\cbox{{green!85!black}}) {d2["A825RY"]}, and state and '+
       f'local government (see\cbox{{purple!70!magenta}}) {d2["A829RY"]}.')

write_txt(text_dir / 'gov.txt', gov)

In [19]:
gov

'Government consumption expeditures and gross investment, which provide services and infrastructure, contributed 0.44 percentage points to real GDP growth in 2019 Q4, compared to an average contribution of 0.52 percentage points over the past year and an average of 0.24 percentage points since 1989. In 2019 Q4, federal defense (see\\cbox{blue!60!black}) contributed 0.17 percentage points, federal nondefense (see\\cbox{green!85!black}) contributed 0.05 percentage points, and state and local government (see\\cbox{purple!70!magenta}) contributed 0.22 percentage points.'

In [28]:
result = df[n.keys()]

data = result.iloc[-5:].iloc[::-1].T

cols = [f' {q.year} Q{q.quarter}' 
        if i == 0 else f'`{str(q.year)[2:]} Q{q.quarter}'
        for i, q in enumerate(data.columns)]

data.columns = cols
data['3-year'] = result.rolling(13).mean().iloc[-1].round(2)
data['10-year'] = result.rolling(41).mean().iloc[-1].round(2)
data['30-year'] = result.rolling(121).mean().iloc[-1].round(2)
data.index = data.index.map(n)
data = data.applymap('{:.2f}'.format)
data.to_csv(data_dir / 'gov.tex', sep='&', line_terminator='\\\ ', quotechar=' ')

### Government receipts and expenditures

In [20]:
# State and local government
s = ['W024RC', 'W023RC']

df = nipa_df(retrieve_table('T30300')['Data'], s)

gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']

df['GDP'] = gdp

(df.div(df['GDP'], axis=0) * 100).loc['1989':].to_csv(data_dir / 'slggdp.csv', index_label='date', float_format='%g')
df = df.dropna() / 1000000

In [22]:
ldate = dtxt(df.index[-1])['qtr1']
gdp = df.GDP.iloc[-1]
gdp3y = df.GDP.iloc[-13]
gdpgr = ((gdp / gdp3y) - 1) * 100
exp = df.W024RC.iloc[-1]
exp3y = df.W024RC.iloc[-13]
expsh = exp / gdp * 100
exp3ysh = exp3y / gdp3y * 100
expgr = ((exp / exp3y) - 1) * 100
rec = df.W023RC.iloc[-1]
rec3y = df.W023RC.iloc[-13]
recsh = rec / gdp * 100
rec3ysh = rec3y / gdp3y * 100
recgr = ((rec / rec3y) - 1) * 100
diff = rec - exp
diffsh = diff / gdp * 100
diff3y = rec3y - exp3y
diffgr = ((diff / diff3y) - 1) * 100
if diff < 0:
    txt = 'deficit'
else:
    txt = 'surplus'

recshch = recsh - rec3ysh
rectxt = f'{["increased" if recshch >= 0 else "decreased"][0]} by a total of {abs(recshch):.2f} percentage points'

expshch = expsh - exp3ysh
exptxt = f'{["increased" if expshch >= 0 else "decreased"][0]} by a total of {abs(expshch):.2f} percentage points'

diffch = recshch - expshch

difftxt = [f'shrink by {diffch:.2f} percent of GDP' if diffch >= 0.1 
             else f'widen by {abs(diffch):.2f} percent of GDP' if diffch <= -0.1 
             else 'be unchanged'][0]


text = (f'Consolidated state and local government expenditures total \${exp:.1f} trillion, '+
        f'or {expsh:.1f} percent of GDP, in {ldate}, and receipts total '+
        f'\${rec:.1f} trillion, equivalent to {recsh:.1f} percent of GDP. The combined state'+
        f' and local government {txt} was '+
        f'\${abs(diff) * 1000:.0f} billion or {abs(diffsh):.2f} percent of GDP. '+
        f'Over the past three years, the expenditures to GDP ratio {exptxt} at the '+
        f'consolidated state and local level, and the ratio of '+
        f'receipts to GDP has {rectxt}, causing the {txt} to {difftxt}.')

write_txt(text_dir / 'govexprec1.txt', text)

text

'Consolidated state and local government expenditures total \\$3.0 trillion, or 13.8 percent of GDP, in 2019 Q4, and receipts total \\$2.8 trillion, equivalent to 12.7 percent of GDP. The combined state and local government deficit was \\$232 billion or 1.07 percent of GDP. Over the past three years, the expenditures to GDP ratio decreased by a total of 0.57 percentage points at the consolidated state and local level, and the ratio of receipts to GDP has decreased by a total of 0.38 percentage points, causing the deficit to shrink by 0.19 percent of GDP.'

In [32]:
# Federal government
s = ['W005RC', 'W013RC']

df = nipa_df(retrieve_table('T30200')['Data'], s)

gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']

df['GDP'] = gdp

(df.div(df['GDP'], axis=0) * 100).to_csv(data_dir / 'fedgdp.csv', index_label='date', float_format='%g')
df = (df / 1000000).dropna()

In [33]:
ldate = dtxt(df.index[-1])['qtr1']
gdp = df.GDP.iloc[-1]
gdp3y = df.GDP.iloc[-13]
gdpgr = ((gdp / gdp3y) - 1) * 100
exp = df.W013RC.iloc[-1]
exp3y = df.W013RC.iloc[-13]
expsh = exp / gdp * 100
exp3ysh = exp3y / gdp3y * 100
expgr = ((exp / exp3y) - 1) * 100
rec = df.W005RC.iloc[-1]
rec3y = df.W005RC.iloc[-13]
recsh = rec / gdp * 100
rec3ysh = rec3y / gdp3y * 100
recgr = ((rec / rec3y) - 1) * 100
diff = rec - exp
diffsh = diff / gdp * 100
diff3y = rec3y - exp3y
diffgr = ((diff / diff3y) - 1) * 100
if diff < 0:
    txt = 'deficit'
else:
    txt = 'surplus'

recshch = recsh - rec3ysh
rectxt = f'{["increased" if recshch >= 0 else "decreased"][0]} by a total of {abs(recshch):.1f} percentage points'

expshch = expsh - exp3ysh
exptxt = f'{["increased" if expshch >= 0 else "decreased"][0]} by a total of {abs(expshch):.1f} percentage points'

diffch = recshch - expshch

difftxt = [f'shrink by {diffch:.1f} percent of GDP' if diffch >= 0.1 
             else f'widen by {abs(diffch):.1f} percent of GDP' if diffch <= -0.1 
             else 'be unchanged'][0]

text = (f'In {ldate}, federal government expenditures total \${exp:.1f} trillion, '+
        f'equivalent to {expsh:.1f} percent of GDP, and receipts total '+
        f'\${rec:.1f} trillion, or {recsh:.1f} percent of GDP. The federal {txt} was therefore '+
        f'\${abs(diff):.1f} trillion or {abs(diffsh):.1f} percent of GDP. '+
        f'Over the past three years, the ratio of expenditures to GDP {exptxt}, and the ratio of '+
        f'receipts to GDP has {rectxt}, causing the {txt} to {difftxt}.')

write_txt(text_dir / 'govexprec2.txt', text)

text

'In 2019 Q4, federal government expenditures total \\$4.9 trillion, equivalent to 22.4 percent of GDP, and receipts total \\$3.6 trillion, or 16.8 percent of GDP. The federal deficit was therefore \\$1.2 trillion or 5.7 percent of GDP. Over the past three years, the ratio of expenditures to GDP increased by a total of 0.4 percentage points, and the ratio of receipts to GDP has decreased by a total of 1.6 percentage points, causing the deficit to widen by 2.0 percent of GDP.'

### Public Debt by Holder

In [34]:
series = ['FDHBATN', 'GFDEBTN', 'FDHBFRBN', 'FDHBPIN', 'FDHBFIN']
start = '1988-01-01'
ftype = '&file_type=json'
base = 'https://api.stlouisfed.org/fred/series/observations?'

df = pd.DataFrame()

for srs in series:
    param = f'series_id={srs}&observation_start={start}&api_key={fred_key}'
   

    url = f'{base}{param}{ftype}'
    r = requests.get(url).json()['observations']
    data = pd.Series({i['date']: (float(i['value']) / 1000.0) if srs in series[:2] else float(i['value']) for i in r})
    
    df[srs] = data

In [35]:
df['PD'] = df['FDHBPIN'] - df['FDHBFIN']
df['IG'] = df['GFDEBTN'] - (df['FDHBFRBN'] + df['FDHBPIN'])

gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']

data = df.div(gdp / 1000.0, axis=0).dropna()

(data[['PD', 'FDHBFIN', 'FDHBFRBN', 'IG']] * 100).to_csv(data_dir / 'pubdebt.csv', index_label='date')

ld = pd.to_datetime(df.index[-1])
ldate = dtxt(ld)['qtr2']

sh = df.div(df['GFDEBTN'], axis=0).iloc[-1] * 100
lv = df.iloc[-1] / 1000
dl = data.iloc[-1] * 100

text = (f'In {ldate}, total public debt was \${lv.GFDEBTN:.1f} trllion, '+
        f'equivalent to {dl.GFDEBTN:.1f} percent of GDP. Of this, \${lv.PD:.1f} '+
        f'trillion, or {sh.PD:.1f} percent of the total, is held by '+
        'private domestic investors (see\cbox{green!60!black}). An additional '+
        f'\${lv.FDHBFIN:.1f} trillion, or {sh.FDHBFIN:.1f} percent '+
        'of the total, is held by foreign investors (see\cbox{orange!70!white}). '+
        'The remainder is held by the Federal Reserve (see\cbox{blue}) '+
        'and various government agencies and trusts (see\cbox{cyan!50!white}), '+
        'such as the Social Security Trust Fund. ')

write_txt(text_dir / 'pubdebt.txt', text)

text

'In the fourth quarter of 2019, total public debt was \\$23.2 trllion, equivalent to 105.5 percent of GDP. Of this, \\$nan trillion, or nan percent of the total, is held by private domestic investors (see\\cbox{green!60!black}). An additional \\$nan trillion, or nan percent of the total, is held by foreign investors (see\\cbox{orange!70!white}). The remainder is held by the Federal Reserve (see\\cbox{blue}) and various government agencies and trusts (see\\cbox{cyan!50!white}), such as the Social Security Trust Fund. '

### Personal Income

In [36]:
s = ['DPCERG']

d = nipa_df(retrieve_table('T20304')['Data'], s)['DPCERG']
deflator = d.iloc[-1] / d

s = ['A065RC', 'A033RC', 'A041RC', 'A048RC', 'W210RC', 'A577RC', 'A061RC']

df = (nipa_df(retrieve_table('T20100')['Data'], s)
      .assign(CAPITAL = lambda x: x['A041RC'] + x['A048RC'] + x['W210RC'],
              TRANSFER = lambda x: x['A577RC'] - x['A061RC'])
      .drop(['A061RC', 'A041RC', 'A048RC', 'W210RC', 'A577RC'], axis=1)
      .multiply(deflator, axis=0))
growth_contrib(df, 'A065RC').loc['1989':].to_csv(data_dir / 'pi.csv', index_label='date')


data = growth_contrib(df, 'A065RC').rename({'A065RC': 'TOTAL', 'A033RC': 'LABOR'}, axis=1)

val3y = data.rolling(12).mean().iloc[-1]

ltdate = dtxt(data.index[-1])['qtr1']


d = {}

for i in data.keys():
    val = data[i].iloc[-1]
    if val >= 0.1:
        d[i] = f'contributed {val:.2f} percentage points to'
    elif val <= 0.1:
        d[i] = f'subtracted {abs(val):.2f} percentage points from'
    else:
        d[i] = 'did not contribute significantly to'
        
d['TOTAL'] = (d['TOTAL']
              .replace('contributed', 'increased at an annualized rate of')
              .replace('subtracted', 'decreased at an annualized rate of')
              .replace('percentage points', 'percent')
              .replace(' to', '').replace(' from', ''))

for i in ['TRANSFER', 'CAPITAL']:
    d[i] = d[i].replace(' to', '').replace(' from', '')
    
    
d2 = {}

for i in data.keys():
    val = val3y[i]
    if val >= 0.1:
        d2[i] = f'contributed an average of {val:.2f} percentage points'
    elif val <= 0.1:
        d2[i] = f'subtracted an average of {abs(val):.2f} percentage points'
    else:
        d2[i] = 'did not contribute significantly, on average'
        
text = (f'Aggregate real personal income {d["TOTAL"]} in {ltdate}. '+
        f'Labor income {d["LABOR"]} overall growth, '+
        f'capital income {d["CAPITAL"]}, and welfare income {d["TRANSFER"]}. ')

write_txt(text_dir / 'pi.txt', text)

text

'Aggregate real personal income increased at an annualized rate of 1.73 percent in 2019 Q4. Labor income contributed 1.45 percentage points to overall growth, capital income contributed 0.13 percentage points, and welfare income contributed 0.15 percentage points. '

### Consumer Spending Growth Contributions

In [37]:
s = ['B230RC']

population = nipa_df(retrieve_table('T20100')['Data'], s)['B230RC']

s = ['DPCERG']

d = nipa_df(retrieve_table('T20304')['Data'], s)['DPCERG']
deflator = d.iloc[-1] / d

s = ['A067RC', 'A068RC', 'A071RC', 'DPCERC']

df = (nipa_df(retrieve_table('T20100')['Data'], s)
      .assign(OTHER = lambda x: -(x['A068RC'] - x['DPCERC']),
              SAVING = lambda x: -x['A071RC'])
      .drop(['A068RC'], axis=1)
      .divide(population, axis=0)
      .multiply(deflator, axis=0))

data = growth_contrib(df, 'DPCERC').rolling(4).mean()
data3y = growth_contrib(df, 'DPCERC').rolling(12).mean()
data.loc['1989':].to_csv(data_dir / 'pcedecomp.csv', index_label='date')

In [38]:
date = f'{data.index[-1].year} Q{data.index[-1].quarter}'

pce = round(data['DPCERC'].iloc[-1], 1)
if pce > 0:
    pcedir = 'increased'
elif pce < 0:
    pcedir = 'decreased'
else: 
    pcedir = ''
pcetext = f'{pcedir} at an average rate of {abs(pce):.1f} percent' 
if pce == 0:
    pcetext = 'was unchanged'
    
slist = ['A067RC', 'SAVING', 'OTHER']
d = {}
for i in slist:
    d[i] = round(data[i].iloc[-1], 1)
    tname = f'{i}txt'
    if d[i] > 0:
        tmpdir = 'added'
    elif d[i] < 0:
        tmpdir = 'subtracted'
    else:
        tmpdir = ''
    tmptxt = f'{tmpdir} {abs(d[i]):.1f} percentage points'
    if d[i] == 0:
        tmptxt = "didn't affect the total"
    d[tname] = tmptxt

In [39]:
pcetxt1 = (f'Real per capita consumer spending {pcetext} over the '+
           f'four quarters ending {date}. Changes to disposable income '+
           f'{d["A067RCtxt"]}, changes to saving {d["SAVINGtxt"]}, and '+
           f'changes to other outlays {d["OTHERtxt"]}.')

In [40]:
pcetxt2 = ('Over the past three years, real per capita consumer spending '+
           f'growth has averaged {data3y["DPCERC"].iloc[-1]:.1f} percent, '+
           f'with income growth contribuing an average of {data3y["A067RC"].iloc[-1]:.1f} '+
           'percentage points and saving subtracting an average of '+
           f'{abs(data3y["SAVING"].iloc[-1]):.1f} percentage points.')

In [41]:
pcetxt = f'{pcetxt1} {pcetxt2}'

write_txt(text_dir / 'pcedecomp.txt', pcetxt)

### Sectoral Accounts

**NOTE:** Need to convert "deficit", "borrower" etc to parameters!

In [42]:
s = ['W162RC', 'W994RC', 'AD01RC', 'W995RC', 'W996RC', 'AD03RC']
df = (nipa_df(retrieve_table('T50100')['Data'], s).div(
      nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC'], axis=0)) * 100

df[['W995RC', 'W996RC', 'AD03RC']].dropna().to_csv(data_dir / 'sectbal2.csv', index_label='date')

data = pd.DataFrame()
data['PRIV'] = df['W994RC']
data['GOV'] = df['AD01RC']
data['ROW'] = -df['W162RC']
data = data.dropna()
date = dtxt(data.index[-1])['qtr1']

data.dropna().to_csv(data_dir / 'sectbal.csv', index_label='date')

priv_curr = abs(data['PRIV'].iloc[-1])
priv_prev = abs(data.loc['2015-01-01', 'PRIV'])
gov_curr = abs(data['GOV'].iloc[-1])
gov_prev = abs(data.loc['2015-01-01', 'GOV'])
row_curr = abs(data['ROW'].iloc[-1])
row_prev = abs(data.loc['2015-01-01', 'ROW'])

In [43]:
sectbaltxt = (f"In {date}, the US private sector was a net lender (running a surplus) of "+
              f"the equivalent of {priv_curr:.1f} percent of GDP, compared to {priv_prev:.1f} "+
              "percent in 2015 Q1. The rest of the world was a net lender to the US, to the "+
              f"equivalent of {row_curr:.1f} percent of GDP in {date} compared to {row_prev:.1f} "+
              f"percent in 2015 Q1. Balancing these transactions, the government (federal, state, "+
              f"and local combined) was a net borrower (running a deficit) of the equivalent "+
              f"of {gov_curr:.1f} percent of GDP, compared to {gov_prev:.1f} percent in 2015. ")
print(sectbaltxt)

write_txt(text_dir / 'sectbal.txt', sectbaltxt)

In 2019 Q4, the US private sector was a net lender (running a surplus) of the equivalent of 5.4 percent of GDP, compared to 2.3 percent in 2015 Q1. The rest of the world was a net lender to the US, to the equivalent of 2.0 percent of GDP in 2019 Q4 compared to 2.4 percent in 2015 Q1. Balancing these transactions, the government (federal, state, and local combined) was a net borrower (running a deficit) of the equivalent of 7.5 percent of GDP, compared to 4.7 percent in 2015. 


In [44]:
dt = df.dropna().index[-1]
ltdate = dtxt(dt)['qtr1']
hhsh = df.loc[dt, 'W996RC']
pbussh = f"{df.loc[dt, 'W995RC']:.1f} percent of GDP"

if (df.loc[dt, 'W995RC'] > -0.1) & (df.loc[dt, 'W995RC'] < 0.1):
    pbussh = 'unchanged'
    
text = (f'Within the private sector, households were net lenders of the equivalent of {hhsh:.1f} '+
        f'percent of GDP in {ltdate}, while the net financial balance of private '+
        f'businesses--corporate and noncorporate--was {pbussh}.')  

write_txt(text_dir / 'sectbal2.txt', text)

text

'Within the private sector, households were net lenders of the equivalent of 4.5 percent of GDP in 2019 Q4, while the net financial balance of private businesses--corporate and noncorporate--was 0.9 percent of GDP.'

### Current Account Balance

In [45]:
s = ['A191RC']

gdp = nipa_df(retrieve_table('T10105')['Data'], s)

s = ['A124RC', 'A253RC', 'A255RC', 'A646RC', 'B656RC', 'B645RC',
     'A655RC', 'A123RC']

df = (nipa_df(retrieve_table('T40100')['Data'], s)
      .assign(GOODS = lambda x: x['A253RC'] - x['A255RC'],
              SERVICES = lambda x: x['A646RC'] - x['B656RC'],
              INCOME = lambda x: x['B645RC'] - x['A655RC'],
              TRANSFERS = lambda x: - x['A123RC'])
      .drop(s[1:], axis=1).drop_duplicates())

data = (df.div(nipa_df(retrieve_table('T10105')['Data'], ['A191RC']
               )['A191RC'], axis=0).dropna().loc['1989':].multiply(100).round(2))

data.loc['1989':].to_csv(data_dir / 'cab.csv', index_label='date')

cab = abs(data['A124RC'].iloc[-1])
tb = abs(data['GOODS'].iloc[-1])
ld = dtxt(data.index[-1])['qtr1']

text = (f'As of {ld}, the US runs a current account deficit of {cab:.1f} '+
        'percent of GDP, primarily as the result of a trade deficit on '+
        f'goods of {tb:.1f} percent of GDP.')

write_txt(text_dir / 'cab.txt', text)

text

'As of 2019 Q4, the US runs a current account deficit of 2.0 percent of GDP, primarily as the result of a trade deficit on goods of 3.8 percent of GDP.'

### Federal Interest Outlays share of GDP

In [46]:
tot = fred_df('FYOINT')['VALUE']
fy = tot.index[-1].year
pfy = tot.index[-2].year
lttot = tot.iloc[-1] / 1000
prtot = tot.iloc[-2] / 1000

data = fred_df('FYOIGDA188S')['VALUE']

data.to_csv(data_dir / 'fedintexp.csv', index_label='date', header=True)

latest = data.iloc[-1]
pval = data.iloc[-2]
val90s = data.loc['1990':'1999'].mean()


text = (f'The Office of Management and Budget '+
        '\href{https://www.whitehouse.gov/omb/historical-tables/}{reports} '+
        f'federal interest outlays of \${lttot:.1f} billion in fiscal year {fy}, '+
        f'compared to \${prtot:.1f} billion in fiscal year {pfy}. '+
        f'Put into the context of the size of the economy, federal interest '+
        f'outlays in {fy} were equivalent to {latest:.1f} percent of '+
        f'GDP, {pval:.1f} percent of GDP in {pfy}, and an average of {val90s:.1f} '+
        'percent in the 1990s, when interest rates were substantially higher. ')

write_txt(text_dir / 'fedintexp.txt', text)

print(text)

node = end_node(data, 'magenta')
write_txt(text_dir / 'fedintexp_node.txt', node)

The Office of Management and Budget \href{https://www.whitehouse.gov/omb/historical-tables/}{reports} federal interest outlays of \$375.2 billion in fiscal year 2019, compared to \$325.0 billion in fiscal year 2018. Put into the context of the size of the economy, federal interest outlays in 2019 were equivalent to 1.8 percent of GDP, 1.6 percent of GDP in 2018, and an average of 2.9 percent in the 1990s, when interest rates were substantially higher. 


In [ ]:
#nipa_series_codes(retrieve_table('T20100'))

### Corporate Profits Destination

In [47]:
s = ['A032RC', 'A438RC', 'A054RC', 'B056RC', 'A127RC']
cprof = nipa_df(retrieve_table('T11200')['Data'], s) / 1000000

cprof['NNI'] = cprof['A032RC'] - cprof['A438RC']
cprof['TAX'] = cprof['A054RC'] / cprof['NNI'] * 100 
cprof['DIV'] = cprof['B056RC'] / cprof['NNI'] * 100 
cprof['RE'] = cprof['A127RC'] / cprof['NNI'] * 100 

cprof[['TAX', 'DIV', 'RE']].loc['1989':].to_csv(data_dir / 'cprof.csv', index_label='date')

dt = cprof.index[-1]
ltdate = dtxt(cprof.index[-1])['qtr2']
tot = cprof.loc[dt, ['A054RC', 'B056RC', 'A127RC']].sum()
totsh = tot / cprof.loc[dt, 'NNI'] * 100
div = cprof.loc[dt, 'B056RC']
divsh = div / cprof.loc[dt, 'NNI'] * 100
ret = cprof.loc[dt, 'A127RC']
tax = cprof.loc[dt, 'A054RC']

text = (f'In {ltdate}, aggregate corporate profits were \${tot:.2f} trillion, or {totsh:.1f} '+
        f'percent of net national income. Of this, \${div:.2f} trillion, equivalent to {divsh:.1f} '+
        'percent of net national product, were paid out as dividends (see\cbox{blue!70!purple}), '+
        f'\${ret*1000:.0f} billion were retained (see\cbox{{cyan!50!white}}), and \${tax*1000:.0f} billion '+
        'went to corporate income tax (see\cbox{red!80!orange}). ')

write_txt(text_dir / 'cprof.txt', text)

### Corporate profits source

In [48]:
s = ['W170RC', 'A262RC', 'W986RC', 'A922RC']
df1 = nipa_df(retrieve_table('T50100')['Data'], s)

s = ['A123RC']
df2 = nipa_df(retrieve_table('T40100')['Data'], s)

s = ['A001RC']
df3 = nipa_df(retrieve_table('T10705')['Data'], s)

cprof = pd.DataFrame()
cprof['ROW Saving'] = (df2['A123RC'] / df3['A001RC']) * 100
cprof['HH Saving'] = (- df1['W986RC'] / df3['A001RC']) * 100
cprof['Gov Saving'] = (- df1['A922RC'] / df3['A001RC']) * 100
cprof['Investment'] = ((df1['W170RC'] - df1['A262RC']) / df3['A001RC']) * 100

cprof.loc['1989':].to_csv(data_dir / 'cprof2.csv', index_label='date')

In [ ]:
#nipa_series_codes(retrieve_table('T50100'))

### Labor Productivity

In [49]:
# Series stored as a dictionary
series = {'PRS85006092': 'value',
          'PRS85006032': 'hours',
          'PRS85006042': 'output'}

# Start year and end year
dates = (1989, 2019)
df = bls_api(series, dates, bls_key)

Post Request Status: REQUEST_SUCCEEDED


In [50]:
df.to_csv(data_dir / 'lprod.csv', index_label='date')

d = series_info(df['value'])

s = {}

for srs in list(series.values()):
    s[srs] = {}
    tmp = series_info(df[srs])
    for i in ['val_latest', 'val_prev']:
        if tmp[i] > 0:
            s[srs][i] = f'increased at an annual rate of {tmp[i]:.1f} percent'
            s[srs][i+'2'] = f'an increase of {tmp[i]:.1f} percent'
        elif tmp[i] < 0:
            s[srs][i] = f'decreased at an annual rate of {abs(d["val_latest"]):.1f} percent'
            s[srs][i+'2'] = f'a decrease of {abs(tmp[i]):.1f} percent'
        else:
            s[srs][i] = 'was unchanged'
            s[srs][i+'2'] = 'no change'   
    

text = (f'In {d["date_latest_ft"]}, labor productivity {s["value"]["val_latest"]} '+
        f'(see\cbox{{teal}}), as the result of {s["output"]["val_latest2"]} in real ouput and '+
        f'{s["hours"]["val_latest2"]} in hours worked. In the prior quarter, '+
        f'{d["date_prev_ft"]}, labor productivity {s["value"]["val_prev"]}, as '+
        f'real output {s["output"]["val_prev"]} and hours of work {s["hours"]["val_prev"]}. '+
        f'Over the past five years, '+
        f'labor productivity growth has averaged {d["five_year_mean"]:.1f} percent, '+
        f'compared to a 1989-onward average of {d["mean"]:.1f} percent.')

write_txt(text_dir / 'lprod.txt', text)


### Gross Labor Income

In [51]:
# Series stored as a dictionary
series = {'LNS12005054': 'Hours', 'LNS12000000': 'Employment'}

# Start year and end year
dates = (1989, 2019)
emp_hrs = bls_api(series, dates, bls_key)

Post Request Status: REQUEST_SUCCEEDED


In [52]:
emp_hrs['Total'] = emp_hrs['Hours'] * emp_hrs['Employment']
emp = emp_hrs['Total'].resample('QS').mean()

s = ['A033RC']
coe = nipa_df(retrieve_table('T20100')['Data'], s)

s = ['DPCERG']
pce = nipa_df(retrieve_table('T20304')['Data'], s)

data = coe.join(pce).join(emp).dropna()
data['real_coe'] = data['A033RC'] / (data['DPCERG'] / data['DPCERG'].iloc[-1])
data['coe_inp'] = data['real_coe'] / data['Total']
data['wage'] = data['coe_inp'] * data['Total'].iloc[0]
data['work'] = data['real_coe'] - data['wage']
# Calculate contributions to growth
result = growth_contrib(data, 'real_coe')[['work', 'wage']]
result.dropna().loc['1989':].to_csv(data_dir / 'gli.csv', index_label='date')

date = f'{result.index[-1].year} Q{result.index[-1].quarter}'
totval = result.iloc[-1].sum()
wage = result['wage'].iloc[-1]
work = result['work'].iloc[-1]

if totval > 0:
    txt1 = f'increased at an annualized and inflation-adjusted rate of {totval:.2f} percent in {date}.'
elif totval < 0:
    txt1 = f'decreased at an annualized and inflation-adjusted rate of {abs(totval):.2f} percent in {date}.'
else:
    txt1 = f'was unchanged, after adjusting for inflation, in {date}.'
    
if wage > 0:
    txt2 = f'Changes in wages contributed {wage:.2f} percentage points'
elif wage < 0:
    txt2 = f'Changes in wages subtracted {abs(wage):.2f} percentage points'
else: 
    txt2 = 'Changes in wages did not contribute'
    
if work > 0:
    txt3 = f', and changes in total hours worked contributed {work:.2f} percentage points.'
elif work < 0:
    txt3 = f', amd changes in total hours worked subtracted {abs(work):.2f} percentage points.'
else: 
    txt3 = ', and changes in total hours worked did not contribute.'    
    
text = f'{txt1} {txt2}{txt3}'
write_txt(text_dir / 'gli.txt', text)

### Shiller real return trailing 20-year average

In [53]:
data = pd.read_excel('http://www.econ.yale.edu/~shiller/data/ie_data.xls', sheet_name='Data')
df = data.iloc[679:1797, np.r_[0, 7, 8]]
df.columns = ['Date', 'Price', 'Dividend']
df.index = pd.to_datetime([f'{val:.2f}' for val in df['Date'].values])
df = df.loc[~df.index.duplicated(keep='first')]

In [54]:
for yrs in [10, 15, 20]:
    s = df.copy()
    mos = yrs * 12
    s['DY'] = (s['Dividend'] / s['Price']).rolling(mos).mean()
    s['Pch'] = s['Price'].pct_change(mos)
    s['Pch'] = (s['Pch']+1)**(1/yrs) - 1
    s = s.dropna()
    s['Return'] = (s['DY'] + s['Pch']) * 100
    s.loc['1989':].dropna().to_csv(data_dir / f'sp500rr{yrs}.csv', 
                                   index_label='date', float_format='%g')
    
text = ('According to historical stock market return '+
        '\href{www.econ.yale.edu/~shiller/data.htm}{data} from Robert Shiller, '+
        'the inflation-adjusted trailing twenty year annual rate of return '+
        f'of the S\&P 500 was {s["Return"].iloc[-1]:.1f} percent as '+
        f'of {s.index[-1].strftime("%B %Y")}. Real returns are currently low relative '+
        'to the average trailing twenty year real annual return of '+
        f'{s["Return"].loc["1995":"2005"].mean():.1f} percent during 1995--2005.')

write_txt(text_dir / 'sp500rr3.txt', text)

text

'According to historical stock market return \\href{www.econ.yale.edu/~shiller/data.htm}{data} from Robert Shiller, the inflation-adjusted trailing twenty year annual rate of return of the S\\&P 500 was 3.8 percent as of December 2019. Real returns are currently low relative to the average trailing twenty year real annual return of 10.1 percent during 1995--2005.'

In [55]:
df = data
df = df.iloc[679:1797, np.r_[0, 7, 8]]
df.columns = ['Date', 'Price', 'Dividend']
df.index = pd.to_datetime([f'{val:.2f}' for val in df['Date'].values])
df = df.loc[~df.index.duplicated(keep='first')]
df['DY'] = (df['Dividend'] / df['Price']).rolling(180).mean()
df['Pch'] = df['Price'].pct_change(180)
df['Pch'] = (df['Pch']+1)**(1/15) - 1
df = df.dropna()
df['Return'] = (df['DY'] + df['Pch']) * 100
df.loc['1989':].dropna().to_csv(data_dir / 'sp500rr4.csv', index_label='date', float_format='%g')

text = ('According to historical stock market return '+
        '\href{www.econ.yale.edu/~shiller/data.htm}{data} from Robert Shiller, '+
        'the inflation-adjusted trailing twenty year annual rate of return '+
        f'of the S\&P 500 was {s["Return"].iloc[-1]:.1f} percent as '+
        f'of {s.index[-1].strftime("%B %Y")}. Real returns are currently low relative '+
        'to the average trailing twenty year real annual return of '+
        f'{s["Return"].loc["1995":"2005"].mean():.1f} percent during 1995--2005.')

write_txt(text_dir / 'sp500rr3.txt', text)

text

'According to historical stock market return \\href{www.econ.yale.edu/~shiller/data.htm}{data} from Robert Shiller, the inflation-adjusted trailing twenty year annual rate of return of the S\\&P 500 was 3.8 percent as of December 2019. Real returns are currently low relative to the average trailing twenty year real annual return of 10.1 percent during 1995--2005.'

In [56]:
df.loc['1989':].dropna()

,Date,Price,Dividend,DY,Pch,Return
1989-01-01,1989.01,610.467,20.9906,0.043979,0.008932,5.291068
1989-02-01,1989.02,626.277,21.0818,0.043963,0.013410,5.737237
1989-03-01,1989.03,619.939,21.2012,0.043956,0.010755,5.471119
1989-04-01,1989.04,636.111,21.2248,0.043934,0.016322,6.025552
1989-05-01,1989.05,656.785,21.3278,0.043899,0.021419,6.531783
...,...,...,...,...,...,...
2019-08-01,2019.08,2925.43,57.3871,0.020168,0.046075,6.624320
2019-09-01,2019.09,3008.55,57.7264,0.020179,0.046360,6.653995
2019-10-01,2019.1,2997.18,57.937,0.020191,0.046491,6.668200
2019-11-01,2019.11,3126.92,58.3105,0.020203,0.046326,6.652872


### Money Growth

In [2]:
base = 'https://www.federalreserve.gov/datadownload/Output.aspx?'
srs = 'rel=H6&series=fafc1295c552e99d2b907eb62278e4ca&lastobs=&'
dt = 'from=01/01/1988&to=12/31/2020&'
oth = 'filetype=csv&label=include&layout=seriescolumn'
url = base + srs + dt + oth

raw_data = pd.read_csv(url)

d = {v: re.sub("\s+[\(\[].*?[\)\]]", "", i) 
     for i, v in raw_data.iloc[4, 1:].iteritems()}

date_column = raw_data.loc[5:, 'Series Description']
date_index = pd.to_datetime(date_column).rename('Date')
columns = raw_data.iloc[4, 1:].values
    
clean_data = raw_data.iloc[5:, 1:].replace('ND', np.nan).astype('float')
clean_data.index = date_index
clean_data.columns = columns

final2 = {}
data = clean_data[['M2_N.WM', 'MMFIN_N.WM']].sum(axis=1)
month_list = clean_data.resample('MS').mean().pct_change(12).dropna().index
short_month = ''
for date in month_list:
    month_len = len(data.loc[date.strftime('%Y-%m')])
    prevyr = f'{date.year - 1}-{date.month}'
    weeks_in_short_month = 4
    end_date = date
    if month_len < 4:
        end_date = date
        short_month = date.strftime('%B %Y')
        val = data.loc[date.strftime('%Y-%m')].mean()
        prv = data.loc[prevyr].iloc[:month_len]
        prev = prv.mean()
        weeks_in_short_month = len(prv)
    elif month_len >= 4:
        val = data.loc[date.strftime('%Y-%m')].mean()
        prev = data.loc[prevyr].mean()
        end_full = date
    final2[date] = (val / prev - 1) * 100
    
final = pd.Series(final2, name='value')
final.to_csv(data_dir / 'm2imf.csv', index_label='date', header='True')

s = series_info(final)

node = end_node(final, 'green!80!blue')
write_txt(text_dir / 'm2imf_node.txt', node)

In [3]:
week_conv = {1: f'the first week of {short_month}', 
             2: f'the first two weeks of {short_month}', 
             3: f'the first three weeks of {short_month}',
             4: end_full.strftime('%B %Y')}

if final.iloc[-1] >= 0.1:
    txt = f'increased over the equivalent previous year value by {final.iloc[-1]:.1f} percent'
elif final.iloc[-1] <= -0.1:
    txt = f'decreased over the equivalent previous year value by {abs(final.iloc[-1]):.1f} percent'   
else:
    txt = 'was virtually unchanged over the previous year value'
    
if s['days_since_match'] > 300:
    txt2 = f", {s['last_matched'].replace('highest level', 'fastest growth rate')}."
else:
    txt2 = '.'
    
text = (f'In {week_conv[weeks_in_short_month]}, '+
        f'the M2 plus institutional money funds measure {txt}{txt2}')

write_txt(text_dir / 'm2imf.txt', text)

gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']
m2sh = (clean_data['M2_N.WM'].iloc[-1] / (gdp.iloc[-1] / 1000)) * 100
imfsh = (clean_data['MMFIN_N.WM'].iloc[-1] / (gdp.iloc[-1] / 1000)) * 100

text2 = (f'In the week of {clean_data.index[-1].strftime("%B %-d, %Y")}, '+
         f'the M2 measure of money averaged \${clean_data["M2_N.WM"].iloc[-1] / 1000:.1f} '+
         f'trillion, equivalent to {m2sh:.1f} percent of GDP. Institution money market '
         +f'accounts, which are not included in M2, can be combined with M2 to create a '+
         f'slightly-broader-than-M2 measure of the money stock. These funds averaged '+
         f'\${clean_data["MMFIN_N.WM"].iloc[-1] / 1000:.1f} trillion in the same week, '+
         f'equivalent to {imfsh:.1f} percent of GDP. ')

write_txt(text_dir / 'm2imf2.txt', text2)
text

'In the first three weeks of March 2020, the M2 plus institutional money funds measure increased over the equivalent previous year value by 9.7 percent, the fastest growth rate since June 2009.'

In [4]:
text2

'In the week of March 16, 2020, the M2 measure of money averaged \\$16.0 trillion, equivalent to 73.5 percent of GDP. Institution money market accounts, which are not included in M2, can be combined with M2 to create a slightly-broader-than-M2 measure of the money stock. These funds averaged \\$2.4 trillion in the same week, equivalent to 11.0 percent of GDP. '